# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [6]:
# Load the CSV file generated from WeatherPy Folder
weather_data = pd.read_csv("weather_df.csv")
dropna_weather_data = weather_data.dropna()
del dropna_weather_data["Unnamed: 0"]
dropna_weather_data.head(20)

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
1,kapaa,US,2020-10-10 12:19:33,22.08,-159.32,20.0,69.0,80.870,5.70
3,chuy,UY,2020-10-10 12:19:33,-33.70,-53.46,0.0,60.0,61.358,2.93
4,ushuaia,AR,2020-10-10 12:19:33,-54.80,-68.30,20.0,37.0,53.870,6.20
5,auki,SB,2020-10-10 12:19:33,-8.77,160.70,97.0,87.0,77.882,1.91
6,khor,RU,2020-10-10 12:19:33,47.89,134.95,41.0,73.0,47.390,2.35
7,khatanga,RU,2020-10-10 12:19:34,71.97,102.50,100.0,98.0,20.102,3.80
8,de-kastri,RU,2020-10-10 12:19:34,51.47,140.78,87.0,77.0,49.298,2.32
10,jamestown,US,2020-10-10 12:17:48,42.10,-79.24,1.0,60.0,73.274,7.20
11,narsaq,GL,2020-10-10 12:19:34,60.92,-46.05,40.0,83.0,50.270,4.10
12,rabaul,PG,2020-10-10 12:19:34,-4.20,152.17,85.0,79.0,79.916,0.66


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [7]:
# Configure gmaps
gmaps.configure(api_key=g_key)

# Locations
locations = dropna_weather_data[["Latitude", "Longitude"]]

humidity = dropna_weather_data["Humidity (%)"].astype(float)

In [8]:
# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=2)

# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [9]:
# Filter vacation with zero cloudiness
vacation_no_cloud = dropna_weather_data[dropna_weather_data["Cloudiness (%)"] == 0]
# Filter vacation with max temp above 70 degrees F
vacation_above_70_degrees = vacation_no_cloud[vacation_no_cloud["Max Temp (F)"] > 70]
# Filter vacation with max temp below 80 degrees F
vacation_below_80_degrees = vacation_above_70_degrees[vacation_above_70_degrees["Max Temp (F)"] < 80]
# Filter vacation with wind speed below 10 mph
vacation_slow_wind = vacation_below_80_degrees[vacation_below_80_degrees["Wind Speed (mph)"] < 10]
# Filter vacation with humidity below 60 %
perfect_vacation = vacation_slow_wind[vacation_slow_wind["Humidity (%)"] < 60]
# Set Index
indexed_perfect_vacation = perfect_vacation.reset_index()
del indexed_perfect_vacation["index"]
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph)
0,mandsaur,IN,2020-10-10 12:18:14,24.07,75.07,0.0,34.0,77.270,1.08
1,castro,BR,2020-10-10 12:19:42,-24.79,-50.01,0.0,36.0,72.410,3.15
2,zhob,PK,2020-10-10 12:19:50,31.34,69.45,0.0,21.0,71.204,3.57
3,saldanha,ZA,2020-10-10 12:19:50,-33.01,17.94,0.0,28.0,70.070,7.70
4,waddan,LY,2020-10-10 12:20:01,29.16,16.14,0.0,33.0,79.268,3.61
5,moron,AR,2020-10-10 12:20:04,-34.65,-58.62,0.0,34.0,72.266,5.10
6,werda,BW,2020-10-10 12:20:09,-25.27,23.28,0.0,21.0,72.410,2.42
7,miram shah,PK,2020-10-10 12:20:24,33.01,70.07,0.0,21.0,73.328,1.47
8,kohlu,PK,2020-10-10 12:20:24,29.90,69.25,0.0,20.0,71.870,1.51
9,marzuq,YE,2020-10-10 12:20:49,14.40,46.47,0.0,26.0,77.162,1.27


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [10]:
vaca_locations = indexed_perfect_vacation[["Latitude", "Longitude"]]

vaca_humidity = indexed_perfect_vacation["Humidity (%)"].astype(float)

# Plot Heatmap
vaca_fig = gmaps.figure()

# Create heat layer
vaca_heat_layer = gmaps.heatmap_layer(vaca_locations, weights=vaca_humidity, 
                                 dissipating=False, max_intensity=50,
                                 point_radius=2.5)

# Add layer
vaca_fig.add_layer(vaca_heat_layer)

# Display figure
vaca_fig

Figure(layout=FigureLayout(height='420px'))

In [19]:
# Hotel variable
hotels = []

# Loop through narrowed down dataframe to get nearest hotel
for city in range(len(indexed_perfect_vacation["City"])):

    lat = indexed_perfect_vacation.loc[city]["Latitude"]
    lng = indexed_perfect_vacation.loc[city]["Longitude"]

    city_coords = f"{lat},{lng}"

    params = {
        "location": city_coords, 
        "types": "lodging",
        "radius": 5000,
        "key": g_key
    }

    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"   

    hotel_request = requests.get(base_url, params=params)
    hotel_response = hotel_request.json()

    try:
        hotels.append(hotel_response["results"][0]["name"])
    except:
        hotels.append("Nearest hotel not found")

# Dataframe with nearest hotel
indexed_perfect_vacation["Nearest Hotel"] = hotels
indexed_perfect_vacation

,City,Country,Date,Latitude,Longitude,Cloudiness (%),Humidity (%),Max Temp (F),Wind Speed (mph),Nearest Hotel
0,mandsaur,IN,2020-10-10 12:18:14,24.07,75.07,0.0,34.0,77.270,1.08,Hotel Rituvan
1,castro,BR,2020-10-10 12:19:42,-24.79,-50.01,0.0,36.0,72.410,3.15,CHACARA BAILLY
2,zhob,PK,2020-10-10 12:19:50,31.34,69.45,0.0,21.0,71.204,3.57,ROOM # 105 DUBAI HOTEL ZHOB
3,saldanha,ZA,2020-10-10 12:19:50,-33.01,17.94,0.0,28.0,70.070,7.70,Blue Bay Lodge
4,waddan,LY,2020-10-10 12:20:01,29.16,16.14,0.0,33.0,79.268,3.61,محل الشريف محمد العايب
5,moron,AR,2020-10-10 12:20:04,-34.65,-58.62,0.0,34.0,72.266,5.10,Hotel Morón
6,werda,BW,2020-10-10 12:20:09,-25.27,23.28,0.0,21.0,72.410,2.42,Nearest hotel not found
7,miram shah,PK,2020-10-10 12:20:24,33.01,70.07,0.0,21.0,73.328,1.47,Bachelor Hostel
8,kohlu,PK,2020-10-10 12:20:24,29.90,69.25,0.0,20.0,71.870,1.51,Captain Shaheed Mir Jahangir Abad kohlu
9,marzuq,YE,2020-10-10 12:20:49,14.40,46.47,0.0,26.0,77.162,1.27,Nearest hotel not found


In [20]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotels = [info_box_template.format(**row) for index, row in indexed_perfect_vacation.iterrows()]
locations = indexed_perfect_vacation[["Lat", "Lng"]]

KeyError: 'Hotel Name'

In [21]:
# Add marker layer ontop of heat map
markers = gmaps.marker_layer(locations, info_box_content= [f"Nearest Hotel: {hotel}" for hotel in hotels])
vacation_figure.add_layer(markers)

# Display figure
vacation_figure

IndexError: list index out of range